The Branch-Bound Algorithm:

In [28]:
import time
import networkx as nx

start_time = time.time()

def is_clique(graph, vertices):
    for v1 in vertices:
        for v2 in vertices:
            if v1 != v2 and (v1, v2) not in graph.edges() and (v2, v1) not in graph.edges():
                return False
    return True

# Figure out whether the graph can be split or not
def recursive_cut(G): 
    G_temp = G.copy()
    clique_limited = 0
    def can_cut(G):
        degrees = G.degree()
        max_degree_node, max_degree = max(degrees, key=lambda x: x[1])
        return max_degree > 0

    # Get the most weighted vertex in the pending graph 
    def get_neighbors_subgraph(G):
        degrees = G.degree()
        max_degree_node, _ = max(degrees, key=lambda x: x[1])
        graph_1,graph_2 = split_graph_by_degree(G,max_degree_node)
        return graph_1,graph_2
    
    # Split the most weighted vertex and its neighbours 
    def split_graph_by_degree(G, node):
        graph_1 = nx.Graph()
        graph_2 = nx.Graph()

        graph_1.add_node(node)
        graph_1.add_nodes_from(G.neighbors(node))
        graph_1.add_edges_from(G.edges(node))

        graph_2 = G.copy()
        graph_2.remove_node(node)
        graph_2.remove_edges_from(G.edges(node))

        return graph_1, graph_2
    subgraphs = []
    while can_cut(G):
        subgraph1,subgraph2 = get_neighbors_subgraph(G)
        if len(subgraph1) == 1:
            break

        # Subgraph with the most wewighted vertex will be added into the list.
        subgraphs.append(subgraph1)        
        if is_clique(G_temp, subgraph1.nodes()):
            clique_limited = len(subgraph1.nodes())
            break

        # Remaining parts of graph will go on pruning.
        G = subgraph2
    return subgraphs, clique_limited

if __name__ == "__main__":
    G = nx.Graph()
    G.add_edges_from([(1,2),(2,3),(2,4),(2,5),(3,4),(3,5),(4,5),(3,6),(6,7),(6,8),(6,9),(6,10),(7,8),(7,9),(7,10),(8,9),(8,10),(9,10)])
    subgraphs, clique_max = recursive_cut(G)
    V = []

    # Traverse all the vertices in the graph
    for i, subgraph in enumerate(subgraphs):
        print(f"Subgraph {i + 1}: {len(subgraph)} Vertices")
        print(subgraph.nodes())
        V = subgraph.nodes()
        subgraph_temp = G.subgraph(subgraph.nodes())
        max_clique = nx.find_cliques(subgraph_temp)
        largest_clique = max(max_clique, key=len)
        print(largest_clique)
        
end_time = time.time()
run_time = end_time - start_time
print("Running Time:", run_time, "sec")

Subgraph 1: 6 Vertices
[6, 3, 7, 8, 9, 10]
[6, 7, 8, 9, 10]
Subgraph 2: 5 Vertices
[2, 1, 3, 4, 5]
[2, 3, 4, 5]
Subgraph 3: 4 Vertices
[7, 8, 9, 10]
[8, 9, 10, 7]
Running Time: 0.0031147003173828125 sec


The Greedy Algorithm:

In [43]:
import time
import networkx as nx

start_t = time.time()

def greedy_max_clique(G):
    clique = []
    for node in G.nodes():
        if all(node in G[clique_node] for clique_node in clique):
            clique.append(node)
    return clique

G = nx.Graph()
G.add_edges_from([(1,2),(2,3),(2,4),(2,5),(3,4),(3,5),(4,5),(3,6),(6,7),(6,8),(6,9),(6,10),(7,8),(7,9),(7,10),(8,9),(8,10),(9,10)])

max_clique = greedy_max_clique(G)

print("The Maximum Clique:", max_clique)
end_t = time.time()
run_time = end_t - start_t
print("Running Time:", run_time, "sec")

The Maximum Clique: [1, 2]
Running Time: 0.0030040740966796875 sec


The program which uses the function in NetworkX:

In [44]:
import time
import networkx as nx

start_x = time.time()

G = nx.Graph()
G.add_edges_from([(1,2),(2,3),(2,4),(2,5),(3,4),(3,5),(4,5),(3,6),(6,7),(6,8),(6,9),(6,10),(7,8),(7,9),(7,10),(8,9),(8,10),(9,10)])

max_clique_size = nx.graph_clique_number(G)

print("Num of vertices in the maximum clique:", max_clique_size)
end_x = time.time()
run_time = end_x - start_x
print("Running Time:", run_time, "sec")

Num of vertices in the maximum clique: 5
Running Time: 0.002791166305541992 sec


The Bron-Kerbosch Algorithm:

In [45]:
import networkx as nx
import time

start_b = time.time()

def bron_kerbosch(graph, R, P, X):
    if not P and not X:
        yield R
    for v in list(P):
        new_R = R + [v]
        new_P = [node for node in P if node in graph.neighbors(v)]
        new_X = [node for node in X if node in graph.neighbors(v)]
        yield from bron_kerbosch(graph, new_R, new_P, new_X)
        P.remove(v)
        X.append(v)

G = nx.Graph()
G.add_edges_from([(1,2),(2,3),(2,4),(2,5),(3,4),(3,5),(4,5),(3,6),(6,7),(6,8),(6,9),(6,10),(7,8),(7,9),(7,10),(8,9),(8,10),(9,10)])

R = []
P = list(G.nodes())
X = []
max_cliques = list(bron_kerbosch(G, R, P, X))
max_clique = max(max_cliques, key=len)
print("The Maximum Clique:", max_clique)
end_b = time.time()
run_time = end_b - start_b
print("Running Time:", run_time, "sec")

The Maximum Clique: [6, 7, 8, 9, 10]
Running Time: 0.0010004043579101562 sec
